In [ ]:
################################# Run simulations for Fig R14-R16 #################################
library(mvtnorm)
library(ggplot2)
set.seed(1)
rm(list = ls())
source("JS_bivariate.R")

nrep <- 50
p <- 10000
p1 <- 1000
p2 <- p - p1
# p1 <- 200
K <- 2
n1 <- 50000
n2 <- 500000

window_size <- p1
nwindow <- p / window_size

rho_Omega_local_all <- 0 #c(0.4,0.8,0.95)
rho_Omega_bg_all <- c(0, 0.4) #c(0,0.4,0.8,0.95)

# per-SNP heritability for global
h2 <- 0.05
omega2_bg <- h2 / p2
omega2_local_all <- omega2_bg * c(1, 3, 5, 10)

omega1_bg_all <- c(0, 0.05) / p2

rho_Sig <- 0
Sigma <- matrix(c(1 / n1, rho_Sig / sqrt(n1 * n2), rho_Sig / sqrt(n1 * n2), 1 / n2), 2, 2)

out_Omega <- data.frame()
out_rho <- data.frame()
out_t1e_all <- data.frame()
pval_pop1 <- data.frame()
pval_pop2 <- data.frame()


for (omega1_bg in omega1_bg_all) {
  for (omega2_local in omega2_local_all) {
    for (rho_Omega_bg in rho_Omega_bg_all) {
      for (rho_Omega_local in rho_Omega_local_all) {

        for (i in 1:nrep) {
          Omega_local <- matrix(c(0, 0, 0, omega2_local), 2, 2)
          Omega_bg <- matrix(c(omega1_bg, rho_Omega_bg * sqrt(omega1_bg * omega2_bg), rho_Omega_bg * sqrt(omega1_bg * omega2_bg), omega2_bg), 2, 2)

          idx1 <- 1:p1
          Beta <- matrix(0, p, K)
          Beta[idx1,] <- rmvnorm(p1, rep(0, K), Omega_local)
          Beta[idx1, 1] <- 0
          Beta[-idx1,] <- rmvnorm(p2, rep(0, K), Omega_bg)
          var(Beta) * p

          idx0 <- which(Beta[, 1] == 0)

          bhat <- Beta + rmvnorm(p, rep(0, K), Sigma)

          pval_marg <- 2 * pnorm(abs(bhat * c(rep(sqrt(n1), p), rep(sqrt(n2), p))), lower.tail = F)

          paras <- estimate_Oemga(bhat, Sigma, verbose = F)
          # OmegaHat*p

          fit_blue <- meta_blue(replicate(p, paras$OmegaHat), replicate(p, Sigma), bhat)

          pval_local <- matrix(rep(0, p * K), p, K)
          for (j in 1:nwindow) {
            idx_window <- (1 + (j - 1) * window_size):(j * window_size)
            paras_j <- estimate_Oemga(bhat[idx_window,], Sigma, verbose = F)
            fit_j <- meta_blue(replicate(window_size, paras_j$OmegaHat), replicate(window_size, Sigma), bhat[idx_window,])
            pval_local[idx_window,] <- fit_j$pval


            if (j == 1) {

              out_Omega <- rbind(out_Omega, data.frame(variance = c(paras_j$OmegaHat0)[c(1, 4, 2)], component = c("omega1", "omega2", "omega12"), rg = rho_Omega_local, rg_bg = rho_Omega_bg, omega1_bg = omega1_bg, omega2_local = omega2_local, method = "local"))
              out_rho <- rbind(out_rho, data.frame(rg = paras_j$OmegaHat[1, 2] / sqrt(paras_j$OmegaHat[1, 1] * paras_j$OmegaHat[2, 2]), rg = rho_Omega_local, rg_bg = rho_Omega_bg, omega1_bg = omega1_bg, omega2_local = omega2_local, method = "local"))
            }
          }


          pval_pop1 <- rbind(pval_pop1, data.frame(pval = pval_marg[, 1], n1 = n1, rg = rho_Omega_local, rg_bg = rho_Omega_bg, omega1_bg = omega1_bg, omega2_local = omega2_local, method = "marginal"))
          pval_pop1 <- rbind(pval_pop1, data.frame(pval = fit_blue$pval[, 1], n1 = n1, rg = rho_Omega_local, rg_bg = rho_Omega_bg, omega1_bg = omega1_bg, omega2_local = omega2_local, method = "global"))
          pval_pop1 <- rbind(pval_pop1, data.frame(pval = pval_local[, 1], n1 = n1, rg = rho_Omega_local, rg_bg = rho_Omega_bg, omega1_bg = omega1_bg, omega2_local = omega2_local, method = "local"))


          pval_pop2 <- rbind(pval_pop2, data.frame(pval = pval_marg[, 2], n1 = n1, rg = rho_Omega_local, rg_bg = rho_Omega_bg, omega1_bg = omega1_bg, omega2_local = omega2_local, method = "marginal"))
          pval_pop2 <- rbind(pval_pop2, data.frame(pval = fit_blue$pval[, 2], n1 = n1, rg = rho_Omega_local, rg_bg = rho_Omega_bg, omega1_bg = omega1_bg, omega2_local = omega2_local, method = "global"))
          pval_pop2 <- rbind(pval_pop2, data.frame(pval = pval_local[, 2], n1 = n1, rg = rho_Omega_local, rg_bg = rho_Omega_bg, omega1_bg = omega1_bg, omega2_local = omega2_local, method = "local"))

          out_t1e_all <- rbind(out_t1e_all, data.frame(t1e = mean(idx0 %in% which(pval_marg[, 1] < 0.05)), rg = rho_Omega_local, rg_bg = rho_Omega_bg, omega1_bg = omega1_bg, omega2_local = omega2_local, method = "marginal"))

          out_t1e_all <- rbind(out_t1e_all, data.frame(t1e = mean(idx0 %in% which(fit_blue$pval[, 1] < 0.05)), rg = rho_Omega_local, rg_bg = rho_Omega_bg, omega1_bg = omega1_bg, omega2_local = omega2_local, method = "global"))

          out_t1e_all <- rbind(out_t1e_all, data.frame(t1e = mean(idx0 %in% which(pval_local[, 1] < 0.05)), rg = rho_Omega_local, rg_bg = rho_Omega_bg, omega1_bg = omega1_bg, omega2_local = omega2_local, method = "local"))


          out_Omega <- rbind(out_Omega, data.frame(variance = c(paras$OmegaHat0)[c(1, 4, 2)], component = c("omega1", "omega2", "omega12"), rg = rho_Omega_local, rg_bg = rho_Omega_bg, omega1_bg = omega1_bg, omega2_local = omega2_local, method = "global"))
          out_rho <- rbind(out_rho, data.frame(rg = paras$OmegaHat[1, 2] / sqrt(paras$OmegaHat[1, 1] * paras$OmegaHat[2, 2]), rg = rho_Omega_local, rg_bg = rho_Omega_bg, omega1_bg = omega1_bg, omega2_local = omega2_local, method = "global"))
        }
        cat("rho_R=", rho_Omega_local, "rho_bg=", rho_Omega_bg, "omega1_bg=", omega1_bg, "omega2_local=", omega2_local, " finished.\n")
      }
    }
  }
}

saveRDS(out_Omega, file = "/home/share/mingxuan/taam/revision/results/Omega_NULL.RDS")
saveRDS(out_rho, file = "/home/share/mingxuan/taam/revision/results/rho_NULL.RDS")
saveRDS(out_t1e_all, file = "/home/share/mingxuan/taam/revision/results/t1e_NULL.RDS")
saveRDS(pval_pop1, file = "/home/share/mingxuan/taam/revision/results/pval1_NULL.RDS")
saveRDS(pval_pop2, file = "/home/share/mingxuan/taam/revision/results/pval2_NULL.RDS")

In [ ]:
################################# Run simulations for Fig R17-R19 #################################
library(mvtnorm)
library(ggplot2)
set.seed(1)
rm(list = ls())
source("JS_bivariate.R")

nrep <- 50
p <- 10000
p1 <- 1000
p2 <- p - p1
# p1 <- 200
K <- 2
n1 <- 50000
n2 <- 500000

prop_nonnull <- 0.1
p1_nonnull <- p1 * prop_nonnull
p2_nonnull <- p2 * prop_nonnull

idx1 <- 1:p1
idx1_nonnull <- 1:p1_nonnull
idx1_null <- (p1_nonnull+1) : p1
idx2_nonnull <- (p1+1) : (p1+p2_nonnull)

window_size <- p1
nwindow <- p / window_size

rho_Omega_local_all <- c(-0.95, -0.8, -0.6, -0.4, -0.2, 0, 0.2, 0.4, 0.6, 0.8, 0.95)
rho_Omega_bg_all <- c(0, 0.2, 0.4, 0.6, 0.8)

# per-SNP heritability for global
h2 <- 0.05
omega_bg <- h2 / p
omega_local_all <- omega_bg # * c(1,3,5,10)


rho_Sig <- 0
Sigma <- matrix(c(1 / n1, rho_Sig / sqrt(n1 * n2), rho_Sig / sqrt(n1 * n2), 1 / n2), 2, 2)

out_Omega <- data.frame()
out_rho <- data.frame()
out_power <- data.frame()


for (omega_local in omega_local_all) {
  for (rho_Omega_bg in rho_Omega_bg_all) {
    for (rho_Omega_local in rho_Omega_local_all) {

      for (i in 1:nrep) {
        Omega_local <- matrix(c(omega_local, rho_Omega_local * omega_local, rho_Omega_local * omega_local, omega_local), 2, 2)
        Omega_bg <- matrix(c(omega_bg, rho_Omega_bg * omega_bg, rho_Omega_bg * omega_bg, omega_bg), 2, 2)


        Beta <- matrix(0, p, K)
        Beta[idx1_nonnull,] <- rmvnorm(p1_nonnull, rep(0, K), Omega_local/prop_nonnull)
        # Beta[idx1,1] <- 0
        Beta[idx2_nonnull,] <- rmvnorm(p2_nonnull, rep(0, K), Omega_bg/prop_nonnull)
        var(Beta) * p

        # idx0 <- which(Beta[, 1] == 0)

        bhat <- Beta + rmvnorm(p, rep(0, K), Sigma)

        pval_marg <- 2 * pnorm(abs(bhat * c(rep(sqrt(n1), p), rep(sqrt(n2), p))), lower.tail = F)

        paras <- estimate_Oemga(bhat, Sigma, verbose = F)
        # OmegaHat*p

        fit_blue <- meta_blue(replicate(p, paras$OmegaHat), replicate(p, Sigma), bhat)

        pval_local <- matrix(rep(0, p * K), p, K)
        for (j in 1:nwindow) {
          idx_window <- (1 + (j - 1) * window_size):(j * window_size)
          paras_j <- estimate_Oemga(bhat[idx_window,], Sigma, verbose = F)
          fit_j <- meta_blue(replicate(window_size, paras_j$OmegaHat), replicate(window_size, Sigma), bhat[idx_window,])
          pval_local[idx_window,] <- fit_j$pval

          if (j == 1) {

            out_Omega <- rbind(out_Omega, data.frame(variance = c(paras_j$OmegaHat0)[c(1, 4, 2)], component = c("omega1", "omega2", "omega12"), rg = rho_Omega_local, rg_bg = rho_Omega_bg, omega_local = omega_local, method = "local"))
            out_rho <- rbind(out_rho, data.frame(rg = paras_j$OmegaHat[1, 2] / sqrt(paras_j$OmegaHat[1, 1] * paras_j$OmegaHat[2, 2]), rg = rho_Omega_local, rg_bg = rho_Omega_bg, omega_local = omega_local, method = "local"))
          }
        }



        out_power <- rbind(out_power, data.frame(power = mean(idx1_nonnull %in% which(pval_marg[, 1] < 0.05)), rg = rho_Omega_local, rg_bg = rho_Omega_bg, omega_local = omega_local, method = "marginal"))

        out_power <- rbind(out_power, data.frame(power = mean(idx1_nonnull %in% which(fit_blue$pval[, 1] < 0.05)), rg = rho_Omega_local, rg_bg = rho_Omega_bg, omega_local = omega_local, method = "global"))

        out_power <- rbind(out_power, data.frame(power = mean(idx1_nonnull %in% which(pval_local[, 1] < 0.05)), rg = rho_Omega_local, rg_bg = rho_Omega_bg, omega_local = omega_local, method = "local"))


        out_Omega <- rbind(out_Omega, data.frame(variance = c(paras$OmegaHat0)[c(1, 4, 2)], component = c("omega1", "omega2", "omega12"), rg = rho_Omega_local, rg_bg = rho_Omega_bg, omega_local = omega_local, method = "global"))
        out_rho <- rbind(out_rho, data.frame(rg = paras$OmegaHat[1, 2] / sqrt(paras$OmegaHat[1, 1] * paras$OmegaHat[2, 2]), rg = rho_Omega_local, rg_bg = rho_Omega_bg, omega_local = omega_local, method = "global"))
      }
      cat("rho_R=", rho_Omega_local, "rho_bg=", rho_Omega_bg, "omega_local=", omega_local, " finished.\n")
    }
  }
}

saveRDS(out_Omega, file = "Omega_nonNULL_sparse.RDS")
saveRDS(out_rho, file = "rho_nonNULL_sparse.RDS")
saveRDS(out_power, file = "power_nonNULL_sparse.RDS")


In [ ]:
################################# Run simulations for Fig R20 #################################
library(mvtnorm)
library(ggplot2)
set.seed(1)
rm(list = ls())
source("JS_bivariate.R")

nrep <- 50
p <- 10000
p1 <- 1000
p2 <- p - p1
# p1 <- 200
K <- 2
n1_all <- c(5000, 10000, 50000, 100000, 150000, 200000)
n2 <- 500000

window_size <- p1
nwindow <- p / window_size

rho_Omega_local_all <- 0 #c(0.4,0.8,0.95)
rho_Omega_bg_all <- 0.4 #c(0,0.4,0.8,0.95)

# per-SNP heritability for global
h2 <- 0.05
omega2_bg <- h2 / p2
omega2_local_all <- omega2_bg * c(10)

omega1_bg_all <- 0.05 / p2

rho_Sig <- 0

out_Omega <- data.frame()
out_rho <- data.frame()
out_power <- data.frame()
out_t1e_all <- data.frame()
pval_pop1 <- data.frame()
pval_pop2 <- data.frame()

for (n1 in n1_all) {
  Sigma <- matrix(c(1 / n1, rho_Sig / sqrt(n1 * n2), rho_Sig / sqrt(n1 * n2), 1 / n2), 2, 2)
  for (omega1_bg in omega1_bg_all) {
    for (omega2_local in omega2_local_all) {
      for (rho_Omega_bg in rho_Omega_bg_all) {
        for (rho_Omega_local in rho_Omega_local_all) {

          for (i in 1:nrep) {
            Omega_local <- matrix(c(0, 0, 0, omega2_local), 2, 2)
            Omega_bg <- matrix(c(omega1_bg, rho_Omega_bg * sqrt(omega1_bg * omega2_bg), rho_Omega_bg * sqrt(omega1_bg * omega2_bg), omega2_bg), 2, 2)

            idx1 <- 1:p1
            Beta <- matrix(0, p, K)
            Beta[idx1,] <- rmvnorm(p1, rep(0, K), Omega_local)
            Beta[idx1, 1] <- 0
            Beta[-idx1,] <- rmvnorm(p2, rep(0, K), Omega_bg)
            var(Beta) * p

            idx0 <- which(Beta[, 1] == 0)

            bhat <- Beta + rmvnorm(p, rep(0, K), Sigma)

            pval_marg <- 2 * pnorm(abs(bhat * c(rep(sqrt(n1), p), rep(sqrt(n2), p))), lower.tail = F)

            paras <- estimate_Oemga(bhat, Sigma, verbose = F)
            # OmegaHat*p

            fit_blue <- meta_blue(replicate(p, paras$OmegaHat), replicate(p, Sigma), bhat)

            pval_local <- matrix(rep(0, p * K), p, K)
            for (j in 1:nwindow) {
              idx_window <- (1 + (j - 1) * window_size):(j * window_size)
              paras_j <- estimate_Oemga(bhat[idx_window,], Sigma, verbose = F)
              fit_j <- meta_blue(replicate(window_size, paras_j$OmegaHat), replicate(window_size, Sigma), bhat[idx_window,])
              pval_local[idx_window,] <- fit_j$pval


              if (j == 1) {

                out_Omega <- rbind(out_Omega, data.frame(variance = c(paras_j$OmegaHat0)[c(1, 4, 2)], component = c("omega1", "omega2", "omega12"), n1 = n1, rg = rho_Omega_local, rg_bg = rho_Omega_bg, omega1_bg = omega1_bg, omega2_local = omega2_local, method = "local"))
                out_rho <- rbind(out_rho, data.frame(rg = paras_j$OmegaHat[1, 2] / sqrt(paras_j$OmegaHat[1, 1] * paras_j$OmegaHat[2, 2]), n1 = n1, rg = rho_Omega_local, rg_bg = rho_Omega_bg, omega1_bg = omega1_bg, omega2_local = omega2_local, method = "local"))
              }
            }


            pval_pop1 <- rbind(pval_pop1,data.frame(pval=pval_marg[, 1], n1 = n1, rg = rho_Omega_local, rg_bg = rho_Omega_bg, omega1_bg = omega1_bg, omega2_local = omega2_local, method = "marginal"))
            pval_pop1 <- rbind(pval_pop1,data.frame(pval=fit_blue$pval[, 1], n1 = n1, rg = rho_Omega_local, rg_bg = rho_Omega_bg, omega1_bg = omega1_bg, omega2_local = omega2_local, method = "global"))
            pval_pop1 <- rbind(pval_pop1,data.frame(pval=pval_local[, 1], n1 = n1, rg = rho_Omega_local, rg_bg = rho_Omega_bg, omega1_bg = omega1_bg, omega2_local = omega2_local, method = "local"))


            pval_pop2 <- rbind(pval_pop2,data.frame(pval=pval_marg[, 2], n1 = n1, rg = rho_Omega_local, rg_bg = rho_Omega_bg, omega1_bg = omega1_bg, omega2_local = omega2_local, method = "marginal"))
            pval_pop2 <- rbind(pval_pop2,data.frame(pval=fit_blue$pval[, 2], n1 = n1, rg = rho_Omega_local, rg_bg = rho_Omega_bg, omega1_bg = omega1_bg, omega2_local = omega2_local, method = "global"))
            pval_pop2 <- rbind(pval_pop2,data.frame(pval=pval_local[, 2], n1 = n1, rg = rho_Omega_local, rg_bg = rho_Omega_bg, omega1_bg = omega1_bg, omega2_local = omega2_local, method = "local"))

            out_t1e_all <- rbind(out_t1e_all, data.frame(t1e = mean(idx0 %in% which(pval_marg[, 1] < 0.05)), n1 = n1, rg = rho_Omega_local, rg_bg = rho_Omega_bg, omega1_bg = omega1_bg, omega2_local = omega2_local, method = "marginal"))

            out_t1e_all <- rbind(out_t1e_all, data.frame(t1e = mean(idx0 %in% which(fit_blue$pval[, 1] < 0.05)), n1 = n1, rg = rho_Omega_local, rg_bg = rho_Omega_bg, omega1_bg = omega1_bg, omega2_local = omega2_local, method = "global"))

            out_t1e_all <- rbind(out_t1e_all, data.frame(t1e = mean(idx0 %in% which(pval_local[, 1] < 0.05)), n1 = n1, rg = rho_Omega_local, rg_bg = rho_Omega_bg, omega1_bg = omega1_bg, omega2_local = omega2_local, method = "local"))

            out_Omega <- rbind(out_Omega, data.frame(variance = c(paras$OmegaHat0)[c(1, 4, 2)], component = c("omega1", "omega2", "omega12"), n1 = n1, rg = rho_Omega_local, rg_bg = rho_Omega_bg, omega1_bg = omega1_bg, omega2_local = omega2_local, method = "global"))
            out_rho <- rbind(out_rho, data.frame(rg = paras$OmegaHat[1, 2] / sqrt(paras$OmegaHat[1, 1] * paras$OmegaHat[2, 2]), n1 = n1, rg = rho_Omega_local, rg_bg = rho_Omega_bg, omega1_bg = omega1_bg, omega2_local = omega2_local, method = "global"))
          }
          cat("rho_R=", rho_Omega_local, "rho_bg=", rho_Omega_bg, "omega1_bg=", omega1_bg, "omega2_local=", omega2_local, " finished.\n")
        }
      }
    }
  }
}
saveRDS(out_Omega, file = "/home/share/mingxuan/taam/revision/results/Omega_NULL_n.RDS")
saveRDS(out_rho, file = "/home/share/mingxuan/taam/revision/results/rho_NULL_n.RDS")
saveRDS(out_t1e_all, file = "/home/share/mingxuan/taam/revision/results/t1e_NULL_n.RDS")
saveRDS(pval_pop1, file = "/home/share/mingxuan/taam/revision/results/pval1_NULL_n.RDS")
saveRDS(pval_pop1, file = "/home/share/mingxuan/taam/revision/results/pval2_NULL_n.RDS")

In [ ]:
################################# Run simulations for Fig R20 #################################
library(mvtnorm)
library(ggplot2)
set.seed(1)
rm(list = ls())
source("JS_bivariate.R")

nrep <- 50
p <- 10000
p1 <- 1000
p2 <- p - p1
# p1 <- 200
K <- 2
n1 <- 50000
n2 <- 500000

prop_nonnull <- 0.1
p1_nonnull <- p1 * prop_nonnull
p2_nonnull <- p2 * prop_nonnull

idx1 <- 1:p1
idx1_nonnull <- 1:p1_nonnull
idx1_null <- (p1_nonnull+1) : p1
idx2_nonnull <- (p1+1) : (p1+p2_nonnull)

window_size <- p1
nwindow <- p / window_size

rho_Omega_local_all <- c(-0.95, -0.8, -0.6, -0.4, -0.2, 0, 0.2, 0.4, 0.6, 0.8, 0.95)
rho_Omega_bg_all <- c(0, 0.2, 0.4, 0.6, 0.8)

# per-SNP heritability for global
h2 <- 0.05
omega_bg <- h2 / p
omega_local_all <- omega_bg # * c(1,3,5,10)


rho_Sig <- 0
Sigma <- matrix(c(1 / n1, rho_Sig / sqrt(n1 * n2), rho_Sig / sqrt(n1 * n2), 1 / n2), 2, 2)

out_Omega <- data.frame()
out_rho <- data.frame()
out_power <- data.frame()


for (omega_local in omega_local_all) {
  for (rho_Omega_bg in rho_Omega_bg_all) {
    for (rho_Omega_local in rho_Omega_local_all) {

      for (i in 1:nrep) {
        Omega_local <- matrix(c(omega_local, rho_Omega_local * omega_local, rho_Omega_local * omega_local, omega_local), 2, 2)
        Omega_bg <- matrix(c(omega_bg, rho_Omega_bg * omega_bg, rho_Omega_bg * omega_bg, omega_bg), 2, 2)


        Beta <- matrix(0, p, K)
        Beta[idx1_nonnull,] <- rmvnorm(p1_nonnull, rep(0, K), Omega_local/prop_nonnull)
        # Beta[idx1,1] <- 0
        Beta[idx2_nonnull,] <- rmvnorm(p2_nonnull, rep(0, K), Omega_bg/prop_nonnull)
        var(Beta) * p

        # idx0 <- which(Beta[, 1] == 0)

        bhat <- Beta + rmvnorm(p, rep(0, K), Sigma)

        pval_marg <- 2 * pnorm(abs(bhat * c(rep(sqrt(n1), p), rep(sqrt(n2), p))), lower.tail = F)

        paras <- estimate_Oemga(bhat, Sigma, verbose = F)
        # OmegaHat*p

        fit_blue <- meta_blue(replicate(p, paras$OmegaHat), replicate(p, Sigma), bhat)

        pval_local <- matrix(rep(0, p * K), p, K)
        for (j in 1:nwindow) {
          idx_window <- (1 + (j - 1) * window_size):(j * window_size)
          paras_j <- estimate_Oemga(bhat[idx_window,], Sigma, verbose = F)
          fit_j <- meta_blue(replicate(window_size, paras_j$OmegaHat), replicate(window_size, Sigma), bhat[idx_window,])
          pval_local[idx_window,] <- fit_j$pval

          if (j == 1) {

            out_Omega <- rbind(out_Omega, data.frame(variance = c(paras_j$OmegaHat0)[c(1, 4, 2)], component = c("omega1", "omega2", "omega12"), rg = rho_Omega_local, rg_bg = rho_Omega_bg, omega_local = omega_local, method = "local"))
            out_rho <- rbind(out_rho, data.frame(rg = paras_j$OmegaHat[1, 2] / sqrt(paras_j$OmegaHat[1, 1] * paras_j$OmegaHat[2, 2]), rg = rho_Omega_local, rg_bg = rho_Omega_bg, omega_local = omega_local, method = "local"))
          }
        }



        out_power <- rbind(out_power, data.frame(power = mean(idx1_nonnull %in% which(pval_marg[, 1] < 0.05)), rg = rho_Omega_local, rg_bg = rho_Omega_bg, omega_local = omega_local, method = "marginal"))

        out_power <- rbind(out_power, data.frame(power = mean(idx1_nonnull %in% which(fit_blue$pval[, 1] < 0.05)), rg = rho_Omega_local, rg_bg = rho_Omega_bg, omega_local = omega_local, method = "global"))

        out_power <- rbind(out_power, data.frame(power = mean(idx1_nonnull %in% which(pval_local[, 1] < 0.05)), rg = rho_Omega_local, rg_bg = rho_Omega_bg, omega_local = omega_local, method = "local"))

        out_Omega <- rbind(out_Omega, data.frame(variance = c(paras$OmegaHat0)[c(1, 4, 2)], component = c("omega1", "omega2", "omega12"), rg = rho_Omega_local, rg_bg = rho_Omega_bg, omega_local = omega_local, method = "global"))
        out_rho <- rbind(out_rho, data.frame(rg = paras$OmegaHat[1, 2] / sqrt(paras$OmegaHat[1, 1] * paras$OmegaHat[2, 2]), rg = rho_Omega_local, rg_bg = rho_Omega_bg, omega_local = omega_local, method = "global"))
      }
      cat("rho_R=", rho_Omega_local, "rho_bg=", rho_Omega_bg, "omega_local=", omega_local, " finished.\n")
    }
  }
}

saveRDS(out_Omega, file = "/home/share/mingxuan/taam/revision/results/Omega_nonNULL_sparse.RDS")
saveRDS(out_rho, file = "/home/share/mingxuan/taam/revision/results/rho_nonNULL_sparse.RDS")
saveRDS(out_power, file = "/home/share/mingxuan/taam/revision/results/power_nonNULL_sparse.RDS")
